In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes

In [2]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run_output")

In [3]:
# Change this line only when you receive a new run
run_folder = "sisepuede_run_2025-07-29T13;04;41.898821"
file_name  = f"{run_folder}.csv"
file_name

'sisepuede_run_2025-07-29T13;04;41.898821.csv'

In [4]:
# Define the run folder and tableau data path
run = output_data.joinpath(run_folder)
tableau = dir_cur.joinpath("Tableau/data")
os.makedirs(tableau, exist_ok=True)

In [5]:
# Define the region
region = "bulgaria"

In [6]:
# Load the output data
ssp_output = pd.read_csv(run.joinpath(file_name))
ssp_output.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,bulgaria,0,538209.795095,1.197172e+06,248404.520813,647461.783416,835490.205420,455983.298622,603185.977623,...,26.919347,2.434011,2.943939,0.0,7.677104,0.0,0.0,73.589885,155.383145,92.81
1,0,bulgaria,1,536985.123196,1.194448e+06,247839.287629,645988.513588,833589.085474,454945.729374,601813.455377,...,28.656257,2.433019,2.537922,0.0,6.744462,0.0,0.0,60.963475,155.294551,92.81
2,0,bulgaria,2,537553.227591,1.195711e+06,248101.489657,646671.938320,834470.982574,455427.040042,602450.145024,...,28.689877,2.429381,2.877470,0.0,6.898984,0.0,0.0,78.638465,152.655636,92.81
3,0,bulgaria,3,532204.152727,1.183813e+06,245632.685874,640237.046977,826167.343924,450895.184949,596455.295097,...,28.689877,2.429381,4.220194,0.0,6.377591,0.0,0.0,63.624617,165.855860,92.81
4,0,bulgaria,4,532503.729973,1.184479e+06,245770.952295,640597.435844,826632.392790,451148.993450,596791.039254,...,28.689877,2.429381,3.979560,0.0,6.766155,0.0,0.0,71.130025,166.353248,92.81


In [7]:
# Load atribute primary data
df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
df_primary.head()

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,68068,0,6002,0
2,69069,0,6003,0


In [8]:
# Load atribute strategy data
df_strategy =pd.read_csv(run.joinpath("ATTRIBUTE_STRATEGY.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
65,4006,IP:ALL,Sectoral Composite - IPPU,All (unique by transformer) IPPU transformations,TX:IPPU:DEC_CLINKER|TX:IPPU:DEC_DEMAND|TX:IPPU...,0
66,6000,PFLO:INC_HEALTHIER_DIETS,Singleton - Default Value - PFLO: Change diets,NaN,TX:PFLO:INC_HEALTHIER_DIETS,0
67,6001,PFLO:INC_IND_CCS,Singleton - Default Value - PFLO: Industrial c...,NaN,TX:PFLO:INC_IND_CCS,0
68,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0
69,6003,PFLO:NDC,NDC,NDC Pathway,TX:LSMM:INC_MANAGEMENT_POULTRY_STRATEGY_NDC|TX...,0


In [9]:
# Merge the dataframes to include primary and strategy information in the ssp_output
ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(  df_strategy, how='left', on='strategy_id')

In [10]:
# Strategys in the output
ssp_output.value_counts('strategy')

strategy
All Actions         36
NDC                 36
Strategy TX:BASE    36
Name: count, dtype: int64

In [11]:
# Get the list of all fields in the output
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [12]:
# Define subsector emission fields that are not energy related
subsector_emission_fields_non_energy = [f for f in subsector_emission_fields if f not in ['emission_co2e_subsector_total_inen', 'emission_co2e_subsector_total_fgtv', 'emission_co2e_subsector_total_entc']]
subsector_emission_fields_non_energy

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [13]:
# Define year based on time_period
ssp_output['year'] = ssp_output['time_period'] + 2015

In [14]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,emission_co2e_subsector_total_agrc,8.128641
1,0,Strategy TX:BASE,0,2016,emission_co2e_subsector_total_agrc,7.682898
2,0,Strategy TX:BASE,0,2017,emission_co2e_subsector_total_agrc,7.985750
3,0,Strategy TX:BASE,0,2018,emission_co2e_subsector_total_agrc,7.749760
4,0,Strategy TX:BASE,0,2019,emission_co2e_subsector_total_agrc,7.900842


In [15]:
# Clean the 'Subsector' column to remove the prefix and capitalize subsector names
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agrc,8.128641
1,0,Strategy TX:BASE,0,2016,Agrc,7.682898
2,0,Strategy TX:BASE,0,2017,Agrc,7.985750
3,0,Strategy TX:BASE,0,2018,Agrc,7.749760
4,0,Strategy TX:BASE,0,2019,Agrc,7.900842


In [16]:
# List of subsectors in the output
ssp_long.value_counts('Subsector')

Subsector
Agrc    108
Ccsq    108
Entc    108
Fgtv    108
Frst    108
Inen    108
Ippu    108
Lndu    108
Lsmm    108
Lvst    108
Scoe    108
Soil    108
Trns    108
Trww    108
Waso    108
Name: count, dtype: int64

In [17]:
# Define a dictionary to map subsector codes to full names
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               108
Carbon Capture and Sequestration          108
Energy Technology                         108
Forest                                    108
Fugitive Emissions                        108
IPPU                                      108
Industrial Energy                         108
Land Use                                  108
Livestock                                 108
Livestock Manure Management               108
Soil Management                           108
Solid Waste                               108
Stationary Combustion and Other Energy    108
Transportation                            108
Wastewater Treatment                      108
Name: count, dtype: int64

In [18]:
# Rename columns
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agriculture,8.128641
1,0,Strategy TX:BASE,0,2016,Agriculture,7.682898
2,0,Strategy TX:BASE,0,2017,Agriculture,7.985750
3,0,Strategy TX:BASE,0,2018,Agriculture,7.749760
4,0,Strategy TX:BASE,0,2019,Agriculture,7.900842
...,...,...,...,...,...,...
1615,69069,NDC,6003,2046,IPPU,6.870555
1616,69069,NDC,6003,2047,IPPU,6.421118
1617,69069,NDC,6003,2048,IPPU,5.975476
1618,69069,NDC,6003,2049,IPPU,5.534048


In [19]:
# Save the long format data to a CSV file for Tableau
ssp_long.to_csv(tableau.joinpath(f"emissions_{region}_{file_name}.csv"), index=False)